In [62]:
%load_ext blackcellmagic

import requests
import pandas as pd
import re
from time import sleep


In [71]:
url = "https://elephrame.com/gather"
payload = {"page": 5, "contentType": "blm", "search": ""}

header = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Referer": "https://elephrame.com/textbook/BLM",
    "Accept": "text/html, */*; q=0.01",
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12) "
        "AppleWebKit/602.1.50 (KHTML, like Gecko) "
        "Version/10.0 Safari/602.1.50"
    ),
    "Origin": "https://elephrame.com",
}

In [96]:
def scrape_elephrame(html):
    events = []
    for section in html.split('item-chart-header"')[1:]:
        id_no =    "".join(re.findall('id="(.*?)"', section))
        location = "".join(re.findall('item-protest-location">(.*?)<', section))
        size = "".join(re.findall("Participant\(s\):</b> (.*?)<", section))
        date = "".join(re.findall('protest-start">.*?, (.*?)<', section))
        time = "".join(re.findall("<b>Time:<\/b>(.*?)<\/li", section, re.DOTALL))
        description = "".join(
            re.findall("<b>Description:<\/b>(.*?)<\/li", section, re.DOTALL)
        )
        subject = "".join(re.findall("Subject.*?>(.*?)<", section))
        links = re.findall('href="(.*?)"', section)
        events.append((id_no, location, size, date, subject, description, time, links))
    new_df = pd.DataFrame(
        events,
        columns=["ID", "Location", "Size", "Date", "Subject", "Description", "Time", "Links"],
    )
    return new_df

In [105]:
dfs = []

for page in range(0, 40):
    payload["page"] = page
    r = requests.post(url, data=payload, headers=header)
    html = r.text
    new_df = scrape_elephrame(html)
    dfs.append(new_df)
    sleep(3)

In [106]:
ele_df = pd.concat(dfs)

In [107]:
ele_df['Collection'] = 'Elephrame'

In [108]:
ele_df.sample(3)

,ID,Location,Size,Date,Subject,Description,Time,Links,Collection
8,3535,"Terre Haute, IN",100+ demonstrators,"May 30, 2020","George Floyd, Breonna Taylor, Ahmaud Arbery, ...",1st march of day w/ recreation of Floyd's kil...,Afternoon,[https://www.tribstar.com/news/local_news/we-j...,Elephrame
15,2804,"Troy, NY",60 demonstrators,"August 22, 2019","Edson Thevenin, Sgt. Randall French, Police O...",Rally at City Council meeting after police fa...,Evening,[https://www.timesunion.com/news/article/Demos...,Elephrame
4,3296,"Los Angeles, CA",Thousands of demonstrators,"June 4, 2020","George Floyd, Breonna Taylor, Ahmaud Arbery, ...",No details included for this demonstration.,Evening,[https://twitter.com/ryansartor/status/1268724...,Elephrame


In [110]:
ele_df.to_csv('Elephrame.csv', index=False)